## Ensemble methods

In this notebook, we are going to build the following estimators from the training set:
- Random forest regressor (<i>Sklearn</i>)
- Gradient-boosted trees (<i>Apache Spark</i>)

After training the ensemble models will be assessed on an evaluation set.

Let's start with importing all indispensable libraries.

In [1]:
import pandas as pd
import numpy as np
import pickle
import h5py
import joblib
from sklearn.model_selection import train_test_split

Now we will load the normalization parameters, training set, and evaluation and test sets.

In [2]:
with open(r"norm_params.pickle", "rb") as output_file:
    norm_params = pickle.load(output_file)

In [3]:
h5f = h5py.File('training.hdf5', 'r')

In [4]:
eval_test_df = pd.read_csv('round2_competition_data/eval_test/eval_test.csv', nrows=2e+6)

To limit the size of the evaluation and test sets we will take a random sample of <i>eval_test</i> dataframe.

In [5]:
# Take a random sample of eval_test dataframe
eval_test_df = eval_test_df.sample(n=500000)

Next, we will split this sample into the evaluation and test sets.

In [6]:
eval_df, test_df = train_test_split(eval_test_df, test_size=0.4)

We have to separate the independent variables from the dependent variables and perform the normalization.

In [7]:
eval_x = np.array(eval_df[norm_params['input_features']])
eval_y = np.array(eval_df[norm_params['target']])

In [8]:
x_min = np.array([norm_params[col]['min'] for col in norm_params['input_features']])
x_max = np.array([norm_params[col]['max'] for col in norm_params['input_features']])

y_min = np.array([norm_params[col]['min'] for col in norm_params['target']])
y_max = np.array([norm_params[col]['max'] for col in norm_params['target']])

In [9]:
normalize = True

# Normalize the data
if normalize:
    eval_x = (eval_x - x_min) / (x_max - x_min)
    eval_y = (eval_y - y_min) / (y_max - y_min)

Below we will define a function that converts WSG84 coordinates to cartesian ones.

In [10]:
def lla_to_ecef(df):
    """Converts WSG84 coordinates to cartesian ones.
    
    """
    
    # Inverse the normalization
    if normalize:
        df = df * (y_max - y_min) + y_min
    
    latitude = np.radians(df[:, 0])
    longitude = np.radians(df[:, 1])
    altitude = df[:, 2]

    # WSG84 ellipsoid constants
    a = 6378137
    e = 8.1819190842622e-2

    # Prime vertical radius of curvature
    N = a / np.sqrt(1 - e**2 * np.sin(latitude)**2)
    
    x = (N + altitude) * np.cos(latitude) * np.cos(longitude)
    y = (N + altitude) * np.cos(latitude) * np.sin(longitude)
    z = ((1 - e**2) * N + altitude) * np.sin(latitude)

    df = np.hstack([np.expand_dims(x, axis=1), np.expand_dims(y, axis=1), np.expand_dims(z, axis=1)])
    
    return df

### RandomForestRegressor

In this section, we are going to build the RandomForestRegressor from the training set. 

Because we are dealing with huge amounts of data, we will have to train the ensemble on sequential small batches. This can be achieved by setting the <i>warm_start</i> parameter to True so that we will be able to fit and add more estimators to the ensemble, where each tree is trained on a separate batch of data.

More information about this ensemble method can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor).

In [11]:
from sklearn.ensemble import RandomForestRegressor

Now we have to specify the batch size or the number of the estimators. In the first case, the dataset will be split based on the batch size into a number of batches, where each batch corresponds to a single estimator. When we specify the <i>n_estimators</i> parameter then a number of batches will be deducted from the number of trees. 

In [12]:
# Specify batch_size or n_estimators
batch_size = None
n_estimators = 100

h5f_length = h5f['input']['table'].len()

if not batch_size:
    indices = np.linspace(0, h5f_length, n_estimators)
    indices = [int(idx) for idx in indices]
    indices = list(zip(indices[:-1], indices[1:]))
else:
    n_batches = int(h5f_length/batch_size)
    indices = [[i*batch_size,(i+1)*batch_size] for i in range(n_batches+1)]
    indices[-1][-1] = h5f_length

# Instantiate the ensemble model
est = RandomForestRegressor(warm_start=True, n_estimators=1)

for idxs in indices:
    start_idx, end_idx = idxs

    # fetch the input and target data
    x = h5f['input']['table'][start_idx:end_idx]['values_block_0']
    y = h5f['target']['table'][start_idx:end_idx]['values_block_0']
    
    if normalize:    
        x = (x - x_min) / (x_max - x_min)
        y = (y - y_min) / (y_max - y_min)
    
    # Fit the batch of the data into the model
    est.fit(x, y)
    est.n_estimators += 1 # add 1 tree


In [13]:
print('Coefficient of determination (R^2): {:.5f}'.format(est.score(eval_x, eval_y)))

Coefficient of determination (R^2): 0.98791


The coefficient of determination is close to 1, which denotes that the output values are explained very well by the determining (independent) variables.

Next, we will predict the regression targets for the evaluation set and calculate the mean distance error.

In [14]:
pred = est.predict(eval_x)

In [15]:
pred_ecef = lla_to_ecef(pred)
target_ecef = lla_to_ecef(eval_y)

# Calculate the average prediciton - target distance error in kilometers
dist_error = np.sqrt((pred_ecef[:, 0] - target_ecef[:, 0])**2 + (pred_ecef[:, 1] - target_ecef[:, 1])**2 + \
        (pred_ecef[:, 2] - target_ecef[:, 2])**2) / 1000

dist_error = np.mean(np.abs(dist_error))

In [16]:
print('Mean distance error: {:.3f} km'.format(dist_error))

Mean distance error: 71.783 km


Now we can save the model for inference.

In [17]:
joblib.dump(est, 'sklearn_random_forest')

['sklearn_random_forest']

###  Apache Spark GradientBoostedTrees

The second model that will be used is the GradientBoostedTrees ensemble from the <i>Apache Spark</i> framework. Regarding <i>Apache Spark 3.0.0</i>, the aforementioned ensemble of decision trees doesn't support multi-target regression, thus to circumvent that restriction we will create a separate GradientBoostedTrees for each target variable.

We will load all the data using <i>Spark</i>, therefore we can restart the kernel and import all libraries.

In [11]:
from pyspark.ml.regression import GBTRegressor, GBTRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.types import Row 
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import pickle
import os

For preprocessing purposes, we have to separate the independent variables from dependant variables. The simplest way to achieve this is to export dataframes containing those variables to CSV files so that Spark will be able to read them later.

In [12]:
store = pd.HDFStore('training.hdf5')

In [13]:
if not os.path.exists('training_feat_csv') and not os.path.exists('training_target_csv'):
    pd.DataFrame(store['input']).to_csv('training_feat_csv')
    pd.DataFrame(store['target']).to_csv('training_target_csv')

Now we can instantiate the <i>SparkSession</i>. For testing, the application will be running locally with 2 cores, and 6 GB of memory for the driver process. 

In [14]:
spark = SparkSession.builder \
    .master("local[2]") \
    .appName("ads-b machine learning") \
    .config("spark.driver.memory", "6g") \
    .getOrCreate()

In [15]:
# Set number of output partitions
spark.conf.set("spark.sql.shuffle.partitions", 100)

# Set log level
spark.sparkContext.setLogLevel("ERROR")

Subsequently, we will load the CSV files containing input features and target variables.

In [16]:
df_x = spark.read.format("csv") \
    .options(header='True', inferSchema='True') \
    .load("training_feat_csv") 

In [17]:
df_y = spark.read.format("csv") \
    .options(header='True', inferSchema='True') \
    .load("training_target_csv") 

We will utilize the <i>VectorAssembler</i> to merge all features into a single vector column.

In [18]:
vectorAssembler_x = VectorAssembler(inputCols=df_x.columns[1:], outputCol = 'features')
df_x = vectorAssembler_x.transform(df_x)
df_x = df_x.select('_c0', 'features')
df_x.show(5)

+---+--------------------+
|_c0|            features|
+---+--------------------+
|  0|[1325.88,1.005387...|
|  1|[12192.0,2.078536...|
|  2|(41,[0,1,2,3,4,13...|
|  3|[10988.04,1.43566...|
|  4|(41,[0,1,2,3,4,13...|
+---+--------------------+
only showing top 5 rows



Next, we will concatenate dataframes of input features and targets.

In [19]:
df = df_x.join(df_y, on='_c0')

In [20]:
df.show(5)

+---+--------------------+------------------+------------------+-----------+
|_c0|            features|          latitude|         longitude|geoAltitude|
+---+--------------------+------------------+------------------+-----------+
|148|[1280.16,7.764831...|         51.091732| 6.521911599999999|     1219.2|
|229|[11582.4,1.347169...|          49.92778|7.4755460000000005|   11376.66|
|307|[9083.04,4.398826...|          53.68795|       -0.39098403|    8983.98|
|326|[11277.6,2.266288...|         50.756012|          8.111547|   11109.96|
|463|[10363.2,3.180464...|53.063126000000004|         -2.331142|   10256.52|
+---+--------------------+------------------+------------------+-----------+
only showing top 5 rows



Now we can load the evaluation and test sets, and merge all input features into a single vector column using the <i>VectorAssembler</i> object created earlier.

In [21]:
eval_df = spark.read.format("csv") \
    .options(header='True', inferSchema='True') \
    .load("round2_competition_data/eval_test/eval_test.csv") 

In [22]:
x_cols = [col for col in eval_df.columns if col not in ['latitude', 'longitude', 'geoAltitude']]
target_cols = ['latitude', 'longitude', 'geoAltitude']

In [23]:
eval_df = vectorAssembler_x.transform(eval_df)
eval_df = eval_df.select('features', 'latitude', 'longitude', 'geoAltitude')
eval_df.show(5)

+--------------------+---------+-----------+-----------+
|            features| latitude|  longitude|geoAltitude|
+--------------------+---------+-----------+-----------+
|[6355.08,5.770127...|52.378876|-0.65986633|     6286.5|
|[9144.0,9.6834109...|  42.2872|  1.7981373|     8915.4|
|(41,[0,1,2,3,4,13...|  47.0522|  5.9036407|    10820.4|
|[10058.4,6.109551...|51.624344|  5.0306993|    9959.34|
|[2125.98,7.921691...|50.820427|  3.6992645|    2080.26|
+--------------------+---------+-----------+-----------+
only showing top 5 rows



In order to be able to join the evaluation dataframe with predictions, we have to create an index.

In [24]:
eval_df = eval_df.withColumn('id', F.monotonically_increasing_id())

In [25]:
eval_df.show(5)

+--------------------+---------+-----------+-----------+---+
|            features| latitude|  longitude|geoAltitude| id|
+--------------------+---------+-----------+-----------+---+
|[6355.08,5.770127...|52.378876|-0.65986633|     6286.5|  0|
|[9144.0,9.6834109...|  42.2872|  1.7981373|     8915.4|  1|
|(41,[0,1,2,3,4,13...|  47.0522|  5.9036407|    10820.4|  2|
|[10058.4,6.109551...|51.624344|  5.0306993|    9959.34|  3|
|[2125.98,7.921691...|50.820427|  3.6992645|    2080.26|  4|
+--------------------+---------+-----------+-----------+---+
only showing top 5 rows



Now we will train and save the ensemble model for each of the target labels separately.

In [26]:
models = {}

for target in target_cols:
    gbt = GBTRegressor(labelCol=target, maxDepth=5, maxBins=128)
    model = gbt.fit(df)
    model.write().overwrite().save('model_{}'.format(target))
    
    models[target] = model

After models are trained we can perform the evaluation.

In [27]:
eval_metrics = {}

for target in target_cols:
    predictions = models[target].transform(eval_df)
    predictions = predictions.withColumnRenamed('prediction', 'pred_{}'.format(target))
    predictions = predictions.select('id', target, 'pred_{}'.format(target))
    
    evaluator = RegressionEvaluator(predictionCol='pred_{}'.format(target), labelCol=target)
    r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
    rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
    
    eval_metrics.setdefault(target, {})
    eval_metrics[target]['rmse'] = rmse
    eval_metrics[target]['r2'] = r2
        
    predictions = predictions.select('id', 'pred_{}'.format(target))
    eval_df = eval_df.join(predictions, on='id')

Now we can print the evaluation metrics.

In [28]:
pd.DataFrame(eval_metrics)

,latitude,longitude,geoAltitude
rmse,0.584128,0.947858,101.624622
r2,0.981343,0.982412,0.999095


Below is the dataframe containing input features, target labels and corresponding predictions.

In [29]:
eval_df.show(5)

+---+--------------------+---------+---------+-----------+------------------+-------------------+------------------+
| id|            features| latitude|longitude|geoAltitude|     pred_latitude|     pred_longitude|  pred_geoAltitude|
+---+--------------------+---------+---------+-----------+------------------+-------------------+------------------+
| 26|[11574.78,7.18157...|54.548813| 9.581694|   11399.52| 54.12998804944908|  9.797703511734095|11443.640799377914|
| 29|[7482.84,6.108366...|51.647003|2.5104501|    7452.36| 51.99551398301082|  2.975453452634862| 7328.026292611733|
| 61|(41,[0,1,2,3,4,13...|51.944897|0.9212036|    7078.98| 51.29541225342674| 1.0365954278826348| 6994.998493908969|
|303|[11574.78,1.09689...|43.607895|1.3043071|   11452.86|43.817816392782696| 2.0580020458729993|11432.630151365685|
|474|[4922.52,7.318009...|51.751877|1.0184623|    4899.66| 50.31336482339997|0.46009835927473464| 4879.642274044956|
+---+--------------------+---------+---------+-----------+------

We will also calculate the mean distance error.

In [30]:
eval_df = eval_df \
    .withColumn('N1', 6378137 / (F.sqrt(1 - 8.1819190842622e-2**2 * F.sin(F.radians(F.col('latitude'))**2)))) \
    .withColumn('N2', 6378137 / (F.sqrt(1 - 8.1819190842622e-2**2 * F.sin(F.radians(F.col('pred_latitude'))**2))))

In [31]:
eval_df = eval_df \
    .withColumn('x1', (F.col('N1') + F.col('geoAltitude')) * F.cos(F.radians(F.col('latitude'))) * F.cos(F.radians(F.col('longitude')))) \
    .withColumn('y1', (F.col('N1') + F.col('geoAltitude')) * F.cos(F.radians(F.col('latitude'))) * F.sin(F.radians(F.col('longitude')))) \
    .withColumn('z1', ((1 - 8.1819190842622e-2**2) * F.col('N1') + F.col('geoAltitude')) * F.sin(F.radians(F.col('latitude')))) \
    .withColumn('x2', (F.col('N2') + F.col('pred_geoAltitude')) * F.cos(F.radians(F.col('pred_latitude'))) * F.cos(F.radians(F.col('pred_longitude')))) \
    .withColumn('y2', (F.col('N2') + F.col('pred_geoAltitude')) * F.cos(F.radians(F.col('pred_latitude'))) * F.sin(F.radians(F.col('pred_longitude')))) \
    .withColumn('z2', ((1 - 8.1819190842622e-2**2) * F.col('N2') + F.col('pred_geoAltitude')) * F.sin(F.radians(F.col('pred_latitude'))))

In [32]:
eval_df = eval_df.withColumn('dist', F.sqrt((F.col('x1') - F.col('x2'))**2 + \
                                           (F.col('y1') - F.col('y2'))**2 + \
                                           (F.col('z1') - F.col('z2'))**2) / 1000)

In [33]:
print('Mean distance error: {:.3f} km' \
      .format(eval_df.select((F.mean(F.abs(F.col('dist'))))).collect()[0][0]))

Mean distance error: 78.506 km
